In [3]:
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Reshape
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from keras.layers.pooling import GlobalAveragePooling2D
from keras.layers import Embedding
from keras import regularizers
from keras.losses import mean_squared_error
import glob
import matplotlib.patches as patches
import json
import numpy as np
from matplotlib.path import Path
import dicom
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

## Open DataSet

In [4]:
case = 'SC-HF-NI-36'

series = json.load(open('series_case.json')) 
image_path_base = './Data/challenge_training/%s/IM-%s' % (case, series[case])
contour_path_base = './Data/Sunnybrook Cardiac MR Database ContoursPart3/\
TrainingDataContours/%s/contours-manual/IRCCI-expert/' % case
contours_list = glob.glob(contour_path_base + '*')
contours_list_series = [k.split('/')[7].split('-')[2] for k in contours_list]

In [5]:
def get_roi(image, contour):
    X_min, Y_min = contour[:,0].min(), contour[:,1].min()
    X_max, Y_max = contour[:,0].max(), contour[:,1].max()  
    w = X_max - X_min
    h = Y_max - Y_min
    mask_roi = np.zeros(image.shape)
    if w > h :
        mask_roi[int(Y_min - (w -h)/2):int(Y_max + (w -h)/2), int(X_min):int(X_max)] = 1.0
    else :
        mask_roi[int(Y_min):int(Y_max), int(X_min - (h-w)/2):int(X_max + (h -w)/2)] = 1.0
    return cv2.resize(mask_roi, (32, 32), interpolation = cv2.INTER_NEAREST)

In [6]:
images, contours = [], []
for c in contours_list_series:
    # Get contours and images path
    idx_contour = contours_list_series.index(c)
    image_path = image_path_base + '-%s.dcm' % c
    contour_path = contours_list[idx_contour]
    
    # open image as numpy array and resize to (64, 64)
    image_part = dicom.read_file(image_path).pixel_array  
    
    # open contours as numpy array
    contour = []
    file = open(contour_path, 'r') 
    for line in file: 
        contour.append(tuple(map(float, line.split())))
    contour = np.array(contour)
    contours.append(get_roi(image_part, contour))
    
    images.append(cv2.resize(image_part, (64, 64)))
    
X = np.reshape(np.array(images), [len(images), 64, 64, 1])
Y = np.reshape(np.array(contours), [len(contours), 1, 32, 32])
print('Dataset shape :', X.shape, Y.shape)

Dataset shape : (33, 64, 64, 1) (33, 1, 32, 32)


In [14]:
#plt.imshow(X[30].reshape(64,64))

In [13]:
#plt.imshow(Y[30].reshape(32, 32))

## Stacked auto-encoder

In [ ]:
inputs = Input(shape=(100, 100))
h = Reshape([64, 64])(inputs)
h = Dense(4096, activation='relu')(h)
encoded = Dense(100, activation='relu')(h)
h = Dense(100, activation='relu')(encoded)
outputs = Dense(4096)(h)

model = Model(input=inputs, output=outputs)
model.compile(optimizer='adam', loss='mse')
model.fit(X, X, batch_size=64, nb_epoch=5)